In [1]:
import datetime
import glob
import HWT_mode_train
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pdb
import pickle
import re
import scalar2vector
import seaborn as sns
import sys
from tensorflow.keras.models import load_model

In [21]:
min_confidence= 3
debug         = True
field         = "REFL_COM"
idir          = "/glade/scratch/ahijevyc"
segmentations = ["hyst"]
nmem          = 10
nns = [f"/glade/work/ahijevyc/NSC_objects/nn_basic.Q1.S1.D1.alllabel.confmin1.ep35.bs10_{i}" for i in range(nmem)]

In [17]:
dfs = pd.DataFrame()
for segmentation in segmentations:
    searchstr = idir+"/track_data_ncarstorm_3km_"+field+"_"+segmentation+"_csv/track_step_NCARSTORM_d01_2*.csv"
    cfiles = glob.glob(searchstr)
    nfiles = len(cfiles)
    if nfiles == 0:
        print("no files match "+searchstr)
        sys.exit(1)

    print("reading",nfiles,"csv files matching "+searchstr) 
    df = [pd.read_csv(cfile, parse_dates=["Run_Date","Valid_Date"]).set_index("Step_ID") for cfile in cfiles]
    df = pd.concat(df)
    df['segmentation'] = segmentation
    dfs = dfs.append(df)

reading 539 csv files matching /glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/track_step_NCARSTORM_d01_2*.csv


In [26]:
HWT_mode_train.label_longname_dict()


{'Q1': 'QLCS\nQ1+Q2+S2', 'S1': 'Supercell\nS1+S3', 'D1': 'Disorganized\nD1+D2'}

In [22]:
# Derive vector magnitude, drop scalar components - already done earlier
# dfs = scalar2vector.scalar2vector(dfs, debug=debug)

# Only keep objects with significant negative or positive UH. Those were the only ones labeled.
dfs=dfs[(dfs.UP_HELI_MAX_max >= 25) | (dfs.UP_HELI_MIN_min <= -25)]

dfs=dfs[(dfs.Forecast_Hour >= 14) & (dfs.Forecast_Hour <=34)]
#dfs = dfs[dfs.Run_Date.dt.year == 2013]

scaler_file = 'scaler.pkl'
if os.path.exists(scaler_file):
    scaler = pickle.load(open(scaler_file,'rb'))
else:
    scaler = dfs.describe()
    pickle.dump(scaler, open(scaler_file,'wb'))
    sys.exit(0)

scaled_dfs = (dfs.select_dtypes(include="number") - scaler.loc["mean"])/scaler.loc["std"]
0,0
features = HWT_mode_train.get_features(suite="basic")



In [23]:
vyyyymmddhh = dfs["Run_Date"].dt.strftime("%Y%m%d%H/").values
preds = []
for nn in nns:
    model = load_model(nn, custom_objects={"brier_score":HWT_mode_train.brier_score, 
                                       "brier_skill_score":HWT_mode_train.brier_skill_score})
    preds.append(model.predict(scaled_dfs[features]))
pred = np.array(preds).mean(axis=0)

In [29]:
labels = HWT_mode_train.label_longname_dict()
labels = labels.keys()
#labels = ["Q1","D1","D2","S1","Q2","S3","S2"]

# TODO: save and restore labels (in correct order). HWT_mode_train.py stores them randomly?


pd.options.display.max_colwidth=200
xx = pd.concat([dfs.reset_index(), pd.DataFrame(pred, columns=labels)], axis='columns')
idir = "https://www2.mmm.ucar.edu/projects/ncar_ensemble/classifymode/mode_img/"
idir = "/glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/"
xx["fn"] = vyyyymmddhh + xx["Step_ID"] + "_+0.png"
for l in labels:
    print(f"mkdir -p {l}")
    local = True
    if local:
        print("cp ", end="")
        print(" ".join(idir + xx.drop(columns="Step_ID").sort_values(by=l, ascending=False).reset_index(drop=True).head(5)["fn"].values)+" ./"+l)
    else:
        print("scp casper.ucar.edu:"+idir+"\{", end="")
        print(",".join(xx.drop(columns="Step_ID").sort_values(by=l, ascending=False).reset_index(drop=True).head(5)["fn"].values)+"\} ./"+l)

mkdir -p Q1
cp /glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/2012060100/d01_REFL_COM_20120601-0000_12_14_000_02_+0.png /glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/2012060100/d01_REFL_COM_20120601-0000_12_14_009_02_+0.png /glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/2012060100/d01_REFL_COM_20120601-0000_12_15_011_02_+0.png /glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/2012060100/d01_REFL_COM_20120601-0000_12_15_011_03_+0.png /glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/2012060100/d01_REFL_COM_20120601-0000_13_14_017_01_+0.png ./Q1
mkdir -p S1
cp /glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/2012060100/d01_REFL_COM_20120601-0000_12_14_000_02_+0.png /glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/2012060100/d01_REFL_COM_20120601-0000_12_14_009_02_+0.png /glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/2012060100/d01_REFL_COM

In [9]:
pwd?

Docstring:
Return the current working directory path.

Examples
--------
::

  In [9]: pwd
  Out[9]: '/home/tsuser/sprint/ipython'
File:      /glade/work/ahijevyc/NSC_objects/20201220_casper_daa/lib/python3.7/site-packages/IPython/core/magics/osm.py


In [28]:
labels

{'Q1': 'QLCS\nQ1+Q2+S2', 'S1': 'Supercell\nS1+S3', 'D1': 'Disorganized\nD1+D2'}